In [2]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [3]:
from utilities import *

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [6]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))

df_icu_graph_features = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_icu_graph_features.pkl','rb'))

df_feats_ppg_train_all = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_train_separate.pkl','rb'))
df_feats_ppg_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_val_separate.pkl','rb'))
df_feats_ppg_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_test.pkl','rb'))


feats_n2v_all_train = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','rb'))
feats_n2v_all_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','rb'))
feats_n2v_all_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','rb'))



Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [7]:
df_feats_clinical_icd = pd.merge(df_feats_clinical,
                                 df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')

In [8]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [9]:
df_icu_graph_features_train = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_val = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_test = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

len(df_icu_graph_features_train)+len(df_icu_graph_features_val)+len(df_icu_graph_features_test)-len(df_icu_graph_features)

0

In [10]:
df_feats_clinical_icd_ppg_icug_train = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_train_all,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_train,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_train,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_icd_ppg_icug_val = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_val,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_icd_ppg_icug_test = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_test,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_clinical_icd_ppg_icug_train)+len(df_feats_clinical_icd_ppg_icug_val)+len(df_feats_clinical_icd_ppg_icug_test)-len(df_feats_clinical_icd)

0

In [45]:
y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

X_train = y_train[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_train,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)
X_val = y_val[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_val,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_test = y_test[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_test,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_train_val = pd.concat([X_train,X_val]).reset_index(drop=True)
y_train_val = pd.concat([y_train,y_val]).reset_index(drop=True)

In [46]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4503),
 (13665, 4503),
 (122983, 4503),
 (13664, 4503),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [47]:
X_train_val.columns.value_counts()

ENCRYPTED_HOSP_ENCOUNTER    1
icd10_transformed_C14       1
icd10_transformed_C20       1
icd10_transformed_C19       1
icd10_transformed_C18       1
                           ..
CPT 1_35560                 1
CPT 1_35563                 1
CPT 1_35565                 1
CPT 1_35566                 1
Node2Vec_127                1
Length: 4503, dtype: int64

# Tuning

In [48]:
del sys.modules["utilities"]
from utilities import *

In [49]:
np.random.seed(666)

In [50]:
a = np.zeros(len(X_train))-1 
b = np.ones(len(X_val)) 
pds = PredefinedSplit(np.concatenate([a,b]))

In [55]:
pipe = Pipeline(steps=[
                       ("imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
                       ("scaler", MinMaxScaler()),
                       ("classifier", MLPClassifier()) #default 3-layer mlp
                        ])

In [56]:
clf = BayesSearchCV(estimator = pipe,
                   search_spaces={
                    'classifier__alpha':Real(0.0001,0.99,'log-uniform'),
                    'classifier__learning_rate': Categorical(['constant','adaptive']),
                    'classifier__early_stopping': Categorical([True,False])
                   },
                   n_iter = 30,
                   cv=pds,
                   scoring = 'roc_auc',
                   n_jobs=32,
                   refit=True,
                   verbose=4,
                   random_state=666
                   ) 

In [57]:
clf.fit(X_train_val.iloc[:,1:],y_train_val.iloc[:,1:].values.ravel())

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


/apps/python/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fitting 1 folds for each of 1 candidates, totalling 1 fits


/apps/python/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV 1/1] END classifier__alpha=0.02345201889819797, classifier__early_stopping=False, classifier__learning_rate=constant;, score=0.838 total time=161.9min
[CV 1/1] END classifier__alpha=0.3290684401395549, classifier__early_stopping=False, classifier__learning_rate=adaptive;, score=0.859 total time=99.9min
[CV 1/1] END classifier__alpha=0.00013416540332600142, classifier__early_stopping=False, classifier__learning_rate=constant;, score=0.768 total time=245.5min
[CV 1/1] END classifier__alpha=0.00010576858387140112, classifier__early_stopping=False, classifier__learning_rate=adaptive;, score=0.790 total time=249.1min
[CV 1/1] END classifier__alpha=0.11592282108997436, classifier__early_stopping=True, classifier__learning_rate=adaptive;, score=0.867 total time= 9.0min
[CV 1/1] END classifier__alpha=0.2165021328936

BayesSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
              estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                        ('scaler', MinMaxScaler()),
                                        ('classifier', MLPClassifier())]),
              n_iter=30, n_jobs=32, random_state=666, scoring='roc_auc',
              search_spaces={'classifier__alpha': Real(low=0.0001, high=0.99, prior='log-uniform', transform='normalize'),
                             'classifier__early_stopping': Categorical(categories=(True, False), prior=None),
                             'classifier__learning_rate': Categorical(categories=('constant', 'adaptive'), prior=None)},
              verbose=4)

In [58]:
print(clf.best_params_)
best_m = clf.best_estimator_
pred = best_m.predict(X_test.iloc[:,1:])
pred_prob = best_m.predict_proba(X_test.iloc[:,1:])[:,1]
print(min(pred_prob),max(pred_prob))

print('Best Validation AUROC: {}'.format(clf.best_score_))

print('Test AUROC: {}'.format(roc_auc_score(y_test.iloc[:,1:], pred_prob)))
print('Test F1-score: {}'.format(f1_score(y_test.iloc[:,1:], pred)))
print('Test Recall: {}'.format(recall_score(y_test.iloc[:,1:], pred)))
print('Test Precision: {}'.format(precision_score(y_test.iloc[:,1:], pred)))
print('Test Accuracy: {}'.format(accuracy_score(y_test.iloc[:,1:], pred)))

OrderedDict([('classifier__alpha', 0.07514813566541474), ('classifier__early_stopping', True), ('classifier__learning_rate', 'adaptive')])
0.00031794923415536403 0.8905875714785064
Best Validation AUROC: 0.8702279137894956
Test AUROC: 0.8494431819130376
Test F1-score: 0.5669217554271435
Test Recall: 0.4601926977687627
Test Precision: 0.7381049206994713
Test Accuracy: 0.797057962529274


In [ ]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [ ]:
#experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[2023])